# Regression for the Prime Counting Function


Model: $y = \dfrac{x}{A \log x + B}$ (nonlinear; linearized via reciprocals to get a linear regression in $A,B$).


## Get Data using Sieve of Eratosthenes


In [15]:
def sieve_bool_array(n: int) -> list[bool]:
    """Return a boolean list where True means prime, up to n."""
    if n < 2:
        return [False] * (n + 1)

    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False

    p = 2
    while p * p <= n:
        if is_prime[p]:
            for multiple in range(p * p, n + 1, p):
                is_prime[multiple] = False
        p += 1
    return is_prime


def prime_pi_data_range(start_x: int, end_x: int, step: int) -> tuple[list[int], list[int]]:
    """
    Generate (x_list, pi_list) where:
      x_list = [start_x, start_x+step, ..., end_x]
      pi_list = [pi(x) for x in x_list]

    Parameters:
        start_x (int): Starting x value (>= 1).
        end_x   (int): Ending x value.
        step    (int): Step size for x values.

    Returns:
        tuple[list[int], list[int]]
    """
    # Safety check
    if start_x < 1:
        start_x = 1
    if step <= 0:
        raise ValueError("Step size must be positive.")
    if end_x < start_x:
        return [], []

    # Step 1: Sieve up to end_x
    is_prime = sieve_bool_array(end_x)

    # Step 2: Precompute pi(x)
    pi = [0] * (end_x + 1)
    count = 0
    for i in range(1, end_x + 1):
        if is_prime[i]:
            count += 1
        pi[i] = count

    # Step 3: Build x_list and pi_list
    x_list = list(range(start_x, end_x + 1, step))
    pi_list = [pi[x] for x in x_list]

    return x_list, pi_list



In [16]:
def count_primes_upto_n(n) -> int:
    return sum(sieve_bool_array(n))

In [17]:
import numpy as np, math
# --- User input ---
x_list, pi_list = prime_pi_data_range(2,10**5,5)  # Input: (start, end, step-size)
#print("x_list:", x_list)
#print("pi_list:", pi_list)        
x = np.array(x_list, dtype=float)
y = np.array(pi_list, dtype=float)
assert np.all(x>1) and np.all(y>0), 'Require x>1 and y>0'


In [18]:
import numpy as np

def legendre_fit_linearized(xs, pis):
    """
    Fit A, B in x/pi(x) ≈ A ln x + B by OLS on (ln x, x/pi(x)),
    using the explicit normal-equation solution:
        beta = (X^T X)^(-1) (X^T y).
    
    Also computes:
      - least-squares error (RSS) in the linearized space y vs A ln x + B,
      - actual errors for the prime counting function on the same x:
          abs_err[i] = pi_hat(x_i) - pi(x_i)
          rel_err[i] = abs_err[i] / pi(x_i)

    Parameters
    ----------
    xs : array-like
        x values (e.g., [10, 100, 1000, 10000]).
    pis : array-like
        corresponding pi(x) values (e.g., [4, 25, 168, 1229]).

    Returns
    -------
    A : float
    B : float
    ls_rss : float
        Residual sum of squares in the linearized fit (y vs A ln x + B).
    pi_abs_err : np.ndarray
        Absolute errors for pi(x): pi_hat(x) - pi_true(x), for the filtered points.
    pi_rel_err : np.ndarray
        Relative errors for pi(x): (pi_hat - pi_true) / pi_true, for the filtered points.

    Notes
    -----
    - Only points with x >= 3 and pi(x) > 0 are used (to avoid log/division issues).
    - The order of error arrays corresponds to the filtered xs.
    - For numerical robustness, QR/SVD or `np.linalg.solve` are preferred in general,
      but here we use the explicit inverse as requested.
    """
    # Convert inputs to 1D float arrays (so Python lists work too)
    xs = np.asarray(xs, dtype=float).ravel()
    pis = np.asarray(pis, dtype=float).ravel()

    if xs.shape != pis.shape:
        raise ValueError(f"xs and pis must have the same shape; got {xs.shape} vs {pis.shape}")

    # Filter valid points
    mask = (xs >= 3) & (pis > 0)
    if mask.sum() < 2:
        raise ValueError("Need at least two valid points with x >= 3 and pi(x) > 0.")

    x = xs[mask]
    pi_true = pis[mask]
    t = np.log(x)
    y = x / pi_true

    # Design matrix: columns [ln x, 1]
    X = np.column_stack([t, np.ones_like(t)])

    # Explicit normal-equation solution: beta = (X^T X)^(-1) (X^T y)
    XtX = X.T @ X
    Xty = X.T @ y

    # Optional: warn if ill-conditioned (we proceed with inverse as requested)
    cond = np.linalg.cond(XtX)
    if not np.isfinite(cond):
        raise np.linalg.LinAlgError("X^T X is singular or not finite.")
    # Proceed with explicit inverse
    beta = np.linalg.inv(XtX) @ Xty

    A, B = float(beta[0]), float(beta[1])

    # Least-squares error (RSS) in the linearized space
    y_hat = X @ beta
    resid = y - y_hat
    ls_rss = float(np.sum(resid**2))

    # Actual errors for pi(x): compare pi_hat(x) vs pi_true(x)
    denom = A * t + B
    pi_hat = x / denom
    pi_abs_err = pi_hat - pi_true
    actual_error = float(np.sum(pi_abs_err**2))
    

    return A, B, ls_rss, actual_error

In [19]:
xs, pis = prime_pi_data_range(2,10000,5)  


A, B, ls_rss, actual_error = legendre_fit_linearized(xs, pis)
print("A=",A," B=",B)
print("Least squares error: ",ls_rss)
print("Actual error: ",actual_error)

A= 0.9373449309795204  B= -0.5071917173860345
Least squares error:  3.0839838412452227
Actual error:  9744.656453499883


In [22]:
# Example prediction at x = 10^10
x_new = 10**10
t_new = np.log(x_new)
y_pred = A * t_new + B
pi_x_pred = x_new/y_pred
print(f"Prediction: pi({x_new}) ≈ {pi_x_pred:.2f}")
print("Actual Correct Value of PrimePi(10^10): ",455052511)

Prediction: pi(10000000000) ≈ 474473943.94
Actual Correct Value of PrimePi(10^10):  455052511
